# This notebook contains the currently working code for the baseline model. The final and ready-to-use version will be in src/baseline.py (at some point)

### Libraries

In [1]:
import torch 
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import os
import numpy as np
from seqeval.metrics import recall_score, precision_score, accuracy_score
from functools import partial

# add the parent directory to the path so we can import the dataloader module
import sys
sys.path.append('..')
from src.data.dataloader import preprocess_data, get_dataset_from_path, get_train_val_test_split
from src.models.utils import compute_weights, weight_to_tensor, get_fbeta_score, compute_metrics
from src.models.trainer import PIITrainer

# Clean Version

In [2]:
#model configuration

class CFG:
    LABELS_LIST = ['B-NAME_STUDENT', 'B-EMAIL', 'B-USERNAME', 'B-ID_NUM', 'B-PHONE_NUM', 'B-URL_PERSONAL', 'B-STREET_ADDRESS', 'I-NAME_STUDENT', 'I-EMAIL', 'I-USERNAME', 'I-ID_NUM', 'I-PHONE_NUM','I-URL_PERSONAL','I-STREET_ADDRESS', 'O']
    label2id = {label: i for i, label in enumerate(LABELS_LIST)}
    label2id['[PAD]'] = -100
    id2label = {i: label for label, i in label2id.items()}
    seed = 42
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.benchmark = True

    # model checkpoint
    model_name = 'bert-base-uncased'
    train_head_only = False

    # path to the directory where the model will be saved
    local_path = os.path.abspath(os.path.abspath(''))
    target_dir = os.path.join(local_path,'..','models', 'baseline')

    #training arguments
    training_args = TrainingArguments(
        output_dir=os.path.join(target_dir, 'trainer'), 
        evaluation_strategy="epoch"
        )

## Loading the model and data

In [3]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)

In [4]:
data_path = os.path.join('..','data', 'raw', 'train.json')
data = get_dataset_from_path(data_path)
data = preprocess_data(data, tokenizer, label2id = CFG.label2id)

Dataset({
    features: ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'],
    num_rows: 6807
})
encoding the labels...


Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

tokenizing and aligning...


Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

flattening the data...


100%|██████████| 6/6 [00:11<00:00,  1.95s/it]


In [5]:
data['labels'][0][:20], tokenizer.decode(data['input_ids'][0][:20])

([-100, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 0, 0, 0, 7, 7, 14, 14, 14],
 '[CLS] design thinking for innovation reflexion - avril 2021 - nathalie sylla challenge & selection')

In [6]:
data_train, data_eval, data_test = get_train_val_test_split(data, seed=CFG.seed)

## Training the model

In [7]:
model = AutoModelForTokenClassification.from_pretrained(CFG.model_name, num_labels=len(CFG.label2id), id2label=CFG.id2label, label2id=CFG.label2id)
trainer = Trainer(
    model=model,
    args=CFG.training_args,
    train_dataset=data_train,
    eval_dataset=data_eval,
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, labels_list=CFG.LABELS_LIST),
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [9]:
trainer.train()

  0%|          | 0/3894 [00:00<?, ?it/s]

Checkpoint destination directory /Users/zofia/Documents/UNI/UPARIS/sem2/pii-projet-tal/notebooks/../models/baseline/trainer/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.0284, 'learning_rate': 4.3579866461222396e-05, 'epoch': 0.39}


Checkpoint destination directory /Users/zofia/Documents/UNI/UPARIS/sem2/pii-projet-tal/notebooks/../models/baseline/trainer/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.0053, 'learning_rate': 3.715973292244479e-05, 'epoch': 0.77}


...

Checkpoint destination directory /Users/zofia/Documents/UNI/UPARIS/sem2/pii-projet-tal/notebooks/../models/baseline/trainer/checkpoint-3500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.0009, 'learning_rate': 5.059065228556755e-06, 'epoch': 2.7}


  0%|          | 0/145 [00:00<?, ?it/s]

{'eval_loss': 0.0009641519282013178, 'eval_recall': 0.8692098092643051, 'eval_precision': 0.7541371158392435, 'eval_fbeta_score': 0.8641383621587829, 'eval_runtime': 45.8054, 'eval_samples_per_second': 25.172, 'eval_steps_per_second': 3.166, 'epoch': 3.0}
{'train_runtime': 4419.0037, 'train_samples_per_second': 7.045, 'train_steps_per_second': 0.881, 'train_loss': 0.005574857275579058, 'epoch': 3.0}


TrainOutput(global_step=3894, training_loss=0.005574857275579058, metrics={'train_runtime': 4419.0037, 'train_samples_per_second': 7.045, 'train_steps_per_second': 0.881, 'train_loss': 0.005574857275579058, 'epoch': 3.0})

In [10]:
model_save_path = os.path.join(CFG.target_dir, 'model')
trainer.save_model(model_save_path)

In [11]:
trainer.evaluate(data_test, metric_key_prefix='test')

  0%|          | 0/161 [00:00<?, ?it/s]

{'test_loss': 0.0019617753569036722,
 'test_recall': 0.8225469728601252,
 'test_precision': 0.7665369649805448,
 'test_fbeta_score': 0.8202418127952598,
 'test_runtime': 51.4568,
 'test_samples_per_second': 24.914,
 'test_steps_per_second': 3.129,
 'epoch': 3.0}

## Loading the model

### From local path

In [12]:
model_from_disk = AutoModelForTokenClassification.from_pretrained(model_save_path)

### From huggingface

In [18]:
token = 'put token here'

model_from_disk.push_to_hub("zmilczarek/pii-detection-baseline-v0.3", token=token)

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zmilczarek/pii-detection-baseline-v0.3/commit/0c04c42385256f6447968aa01be9449b17faca61', commit_message='Upload BertForTokenClassification', commit_description='', oid='0c04c42385256f6447968aa01be9449b17faca61', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
model_from_huggingface = AutoModelForTokenClassification.from_pretrained("zmilczarek/pii-detection-baseline-v0.3")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Evaluating the huggingface model

In [20]:
trainer = Trainer(
    model=model_from_huggingface,
    train_dataset=data_train,
    eval_dataset=data_test,
    compute_metrics=partial(compute_metrics, labels_list = CFG.LABELS_LIST),
)

#eval
trainer.evaluate(data_test, metric_key_prefix='test')

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/161 [00:00<?, ?it/s]

{'test_loss': 0.0019617753569036722,
 'test_recall': 0.8225469728601252,
 'test_precision': 0.7665369649805448,
 'test_fbeta_score': 0.8202418127952598,
 'test_runtime': 50.8393,
 'test_samples_per_second': 25.217,
 'test_steps_per_second': 3.167}

-------------------------

## Training head only 

I needed to change the compute_metrics method into the one below, because otherwise I was getting an error during training. 

In [49]:
def compute_metrics(p, id2label):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    #print(predictions[1])
    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #print(true_predictions[1])
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    #print(true_labels[1])

    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    fbeta_score = get_fbeta_score(precision, recall)

    results = {
        'recall': recall,
        'precision': precision,
        'fbeta_score': fbeta_score
    }
        
    return results

In [50]:
model = AutoModelForTokenClassification.from_pretrained(CFG.model_name, num_labels=len(CFG.label2id), id2label=CFG.id2label, label2id=CFG.label2id)
for param in model.base_model.parameters():
    param.requires_grad = False

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
training_args_head = CFG.training_args
training_args_head.output_dir = os.path.join(CFG.target_dir, 'head_only')
training_args_head.num_train_epochs = 10
trainer_head = Trainer(
    model=model,
    args=training_args_head,
    train_dataset=data_train,
    eval_dataset=data_eval,
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, id2label=CFG.id2label),
)

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [47]:
training_args_head.output_dir

'/Users/zofia/Documents/UNI/UPARIS/sem2/pii-projet-tal/notebooks/../models/baseline/head_only'

The training in the cellbelow was run from a checkpoint

In [52]:
trainer_head.train()

  0%|          | 0/12980 [00:00<?, ?it/s]

Checkpoint destination directory /Users/zofia/Documents/UNI/UPARIS/sem2/pii-projet-tal/notebooks/../models/baseline/head_only/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.7411, 'learning_rate': 4.8073959938366716e-05, 'epoch': 0.39}


Checkpoint destination directory /Users/zofia/Documents/UNI/UPARIS/sem2/pii-projet-tal/notebooks/../models/baseline/head_only/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.0422, 'learning_rate': 4.614791987673344e-05, 'epoch': 0.77}


...

Checkpoint destination directory /Users/zofia/Documents/UNI/UPARIS/sem2/pii-projet-tal/notebooks/../models/baseline/head_only/checkpoint-12500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.005, 'learning_rate': 1.848998459167951e-06, 'epoch': 9.63}


  0%|          | 0/145 [00:00<?, ?it/s]

{'eval_loss': 0.0034655591007322073, 'eval_recall': 0.2615803814713896, 'eval_precision': 0.48, 'eval_fbeta_score': 0.26624, 'eval_runtime': 45.8801, 'eval_samples_per_second': 25.131, 'eval_steps_per_second': 3.16, 'epoch': 10.0}
{'train_runtime': 5570.0518, 'train_samples_per_second': 18.63, 'train_steps_per_second': 2.33, 'train_loss': 0.03775820827631076, 'epoch': 10.0}


TrainOutput(global_step=12980, training_loss=0.03775820827631076, metrics={'train_runtime': 5570.0518, 'train_samples_per_second': 18.63, 'train_steps_per_second': 2.33, 'train_loss': 0.03775820827631076, 'epoch': 10.0})

In [54]:
model_head_save_path = os.path.join(CFG.target_dir, 'head_only', 'model')
trainer_head.save_model(model_head_save_path)

In [53]:
trainer_head.evaluate(data_test, metric_key_prefix='test')

  0%|          | 0/161 [00:00<?, ?it/s]

{'test_loss': 0.00500537920743227,
 'test_recall': 0.21085594989561587,
 'test_precision': 0.39453125,
 'test_fbeta_score': 0.21470035156569375,
 'test_runtime': 50.1787,
 'test_samples_per_second': 25.549,
 'test_steps_per_second': 3.209,
 'epoch': 10.0}

In [56]:
model_from_disk.push_to_hub("zmilczarek/pii-detection-baseline-head-only-v0.1", token=token)

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zmilczarek/pii-detection-baseline-head-only-v0.1/commit/8fc5ae8e2258afac7738ff4a9e1c1d27e46bab0f', commit_message='Upload BertForTokenClassification', commit_description='', oid='8fc5ae8e2258afac7738ff4a9e1c1d27e46bab0f', pr_url=None, pr_revision=None, pr_num=None)

### Loading the model

## Training on 10 epochs

# Other attempts (different compute metrics, different loss function etc)

## Using the model

### Loading the model

In [ ]:
# model_dir = 'model/model_initial_preprocessing'
# model_loaded = AutoModelForTokenClassification.from_pretrained(model_dir)
# model_loaded = model_loaded.to('cpu')

In [22]:
model_loaded = AutoModelForTokenClassification.from_pretrained(model_save_path)

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [23]:
trainer = Trainer(
    model=model_loaded,
    train_dataset=data_train,
    eval_dataset=data_test,
    compute_metrics=partial(compute_metrics, labels_list = CFG.LABELS_LIST),
)

#eval
trainer.evaluate(data_test, metric_key_prefix='test')

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/161 [00:00<?, ?it/s]

{'test_loss': 0.0023406886029988527,
 'test_recall': 0.7954070981210856,
 'test_precision': 0.8141025641025641,
 'test_fbeta_score': 0.7961102627983605,
 'test_runtime': 50.2165,
 'test_samples_per_second': 25.529,
 'test_steps_per_second': 3.206}

In [24]:
token ='put token here'
model_loaded.push_to_hub('zmilczarek/pii-detection-baseline-v0.3', token = token)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zmilczarek/pii-detection-baseline-v0.3/commit/77c400cec12e992133d9528279b271d49480754f', commit_message='Upload BertForTokenClassification', commit_description='', oid='77c400cec12e992133d9528279b271d49480754f', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
model_from_huggingface = AutoModelForTokenClassification.from_pretrained('zmilczarek/pii-detection-baseline-v0.2')

In [13]:
trainer = Trainer(
    model=model_from_huggingface,
    train_dataset=data_train,
    eval_dataset=data_test,
    compute_metrics=partial(compute_metrics, labels_list = CFG.LABELS_LIST),
)

#eval
trainer.evaluate(data_test, metric_key_prefix='test')

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/161 [00:00<?, ?it/s]

{'test_loss': 0.000548205862287432,
 'test_recall': 0.964509394572025,
 'test_precision': 0.9295774647887324,
 'test_fbeta_score': 0.9631173829377806,
 'test_runtime': 50.3708,
 'test_samples_per_second': 25.451,
 'test_steps_per_second': 3.196}

## Training just the head

### Inference

In [ ]:
"""
The plan to make the model label test.csv correctly

1. Load the model
2. Prepare the dataset  (prepare input ids/ att mask in chunks)
3. Get the labels

"""